# Accuracy 

In [ ]:
# Note : Can only do one word per time as our lexicon cover unstemmed words also but not other dictionary except PRPM do that 

In [18]:
from PyClasses.Scrapers.PRPMScraper import PRPMScraper
from PyClasses.Scrapers.WikiScrap import WikiScraper
from PyClasses.Preprocessing.PRPMCleaner import PRPMCleaner
from PyClasses.Neo4j.lexicon_nodes import LexiconNodeManager

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

/home/student/de-prj/de-venv/lib/python3.10/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/student/de-prj/de-venv/lib/python3.10/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [19]:
# Neo4j
uri = "neo4j+s://bb539a73.databases.neo4j.io"
auth= ("neo4j", "nHf-IeFSY8xVQvFU4UEaLs0ugES0uD7R-lR8mGgxt9w")

lnm = LexiconNodeManager(uri, auth)

word_to_find = "buku" 
single_word_query = f"""
    MATCH (n:WORD {{word: '{word_to_find}'}})
    RETURN n.word AS word, n.definitions AS definitions
    LIMIT 1
    """

neoresult = lnm.create_custom_query(single_word_query)
print(neoresult)

Established Connection
Successfully run 
    MATCH (n:WORD {word: 'buku'})
    RETURN n.word AS word, n.definitions AS definitions
    LIMIT 1
    !
[{'word': 'buku', 'definitions': ['berhelai-helai kertas berjilid yang berisi tulisan untuk dibaca atau berisi ruangan-ruangan kosong untuk ditulisi']}]


In [20]:
wk = WikiScraper()
word = 'cari'
wikiresult = wk.findWordMetaData(word)

if wikiresult and 'meanings' in wikiresult[word]:
    raw_meanings = wikiresult[word]['meanings']
    cleaned_meanings = prpm_cleaner.clean_all_meanings(raw_meanings)
    wikiresult[word]['meanings'] = cleaned_meanings

print(wikiresult)

{'cari': {'WikiDefi': ['Berusaha menjumpai sesuatu yang sukar/tidak kelihatan atau hilang.']}}


In [22]:
wiki_meaning = wikiresult[word]['WikiDefi'][0]
print(wiki_meaning)

Berusaha menjumpai sesuatu yang sukar/tidak kelihatan atau hilang.


In [23]:
neo_combined = ' '.join(neoresult[0]['definitions'])
print(neo_combined)

berhelai-helai kertas berjilid yang berisi tulisan untuk dibaca atau berisi ruangan-ruangan kosong untuk ditulisi


In [24]:
import re

def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)  # Remove all punctuation
    text = ' '.join(text.split())       # Normalize multiple spaces to a single space
    return text.lower()                 # Convert to lowercase

# Clean the input texts
wiki_cleaned = clean_text(wiki_meaning)
neo_cleaned = clean_text(neo_combined)

# Tokenize after cleaning
wiki_tokens = set(wiki_cleaned.split())
neo_tokens = set(neo_cleaned.split())

# Calculate intersection and accuracy
intersection = wiki_tokens & neo_tokens
accuracy = len(intersection) / len(wiki_tokens)

# Output
print(f"Token Overlap Accuracy: {accuracy:.2f}")


Token Overlap Accuracy: 0.25


In [ ]:
print(wiki_tokens)

In [ ]:
print(neo_tokens)

# Coverage 

In [ ]:
from PyClasses.Neo4j.lexicon_nodes import LexiconNodeManager

In [ ]:
uri = "neo4j+s://bb539a73.databases.neo4j.io"
auth= ("neo4j", "nHf-IeFSY8xVQvFU4UEaLs0ugES0uD7R-lR8mGgxt9w")

In [ ]:
lnm = LexiconNodeManager(uri, auth)

In [ ]:
analysis_query = """
    MATCH (n:WORD)
    RETURN n.word AS word, n.word_count AS count, n.definitions AS definitions
    ORDER BY n.word_count DESC
    LIMIT 1000
    """

result = lnm.create_custom_query(analysis_query)

# Analyze coverage
total_words = len(result)
covered_words = 0
not_covered_words = []

for word in result:
    definitions = word['definitions']
    if definitions and any(defn.strip() for defn in definitions):
        covered_words += 1
    else: 
        not_covered_words.append(word['word'])

coverage_percentage = (covered_words / total_words) * 100 if total_words > 0 else 0

print(f"Total Words: {total_words}")
print(f"Covered Words: {covered_words}")
print(f"Coverage Percentage: {coverage_percentage:.2f}%")

print("\nExample of Not Covered Words:")
print(", ".join(not_covered_words))

# Consistency 

In [ ]:
# Consistency : 
# If the model consistently labels words as verbs (even when they are not), 
# it shows that the model is following a consistent pattern 
# (predicting verbs) across different inputs.
# Shit accuracy 

In [16]:
lists = ["perempuan", "polis", "guru", "rumah", "buku"]

In [29]:
from PyClasses.Neo4j.lexicon_nodes import LexiconNodeManager

uri = "neo4j+s://bb539a73.databases.neo4j.io"
auth= ("neo4j", "nHf-IeFSY8xVQvFU4UEaLs0ugES0uD7R-lR8mGgxt9w")

lnm = LexiconNodeManager(uri, auth)

for word in lists: 
    single_word_query = f"""
        MATCH (n:WORD {{word: '{word}'}})
        RETURN n.word AS word, n.POS AS pos
        LIMIT 1
        """
    neoresult = lnm.create_custom_query(single_word_query)
    print(neoresult) 

Established Connection
Successfully run 
        MATCH (n:WORD {word: 'perempuan'})
        RETURN n.word AS word, n.POS AS pos
        LIMIT 1
        !
[{'word': 'perempuan', 'pos': 'Proper noun'}]
Successfully run 
        MATCH (n:WORD {word: 'polis'})
        RETURN n.word AS word, n.POS AS pos
        LIMIT 1
        !
[{'word': 'polis', 'pos': 'Proper noun'}]
Successfully run 
        MATCH (n:WORD {word: 'guru'})
        RETURN n.word AS word, n.POS AS pos
        LIMIT 1
        !
[{'word': 'guru', 'pos': 'Proper noun'}]
Successfully run 
        MATCH (n:WORD {word: 'rumah'})
        RETURN n.word AS word, n.POS AS pos
        LIMIT 1
        !
[{'word': 'rumah', 'pos': 'Proper noun'}]
Successfully run 
        MATCH (n:WORD {word: 'buku'})
        RETURN n.word AS word, n.POS AS pos
        LIMIT 1
        !
[{'word': 'buku', 'pos': 'Proper noun'}]


In [ ]:
# 